# Model Implementation

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()

    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),

        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

  def forward(self, x):
    return self.conv(x)

class UNet(nn.Module):
  def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
    super(UNet, self).__init__()

    self.downs = nn.ModuleList()
    self.ups = nn.ModuleList()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    # Downsamplin part of UNet
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

    # Upsampling part of UNet
    for feature in reversed(features):
      self.ups.append(
          nn.ConvTranspose2d(
              feature*2, feature, kernel_size=2, stride=2)
      )
      self.ups.append(DoubleConv(feature*2, feature))

      self.bottleneck = DoubleConv(features[-1], features[-1]*2)
      self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

  def forward(self, x):
    skip_connections = []

    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)

    x = self.bottleneck(x)
    skip_connections = skip_connections[::-1]

    for idx in range(0, len(self.ups), 2):
      x = self.ups[idx](x)
      skip_connection = skip_connections[idx//2]

      if x.shape != skip_connection.shape:
        x = TF.resize(x, size=skip_connection.shape[2:])

      concat_skip = torch.cat((x, skip_connection), dim=1)
      x = self.ups[idx+1](concat_skip)

    return self.final_conv(x)

In [ ]:
def test():
  x = torch.randn((3, 1, 161, 161))
  model = UNet(in_channels=1, out_channels=1)
  preds = model(x)
  print(preds.shape)
  print(x.shape)

  assert preds.shape == x.shape

# Dataset Loading

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset
import numpy as np

class SegmentationDataset(Dataset):
  def __init__(self, image_dir, mask_dir, transform=None):
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transform
    self.images = os.listdir(image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    img_path = os.path.join(self.image_dir, self.images[index])
    mask_path = os.path.join(self.mask_dir, self.images[index].replace('.jpg', '.png'))

    image = np.array(Image.open(img_path).convert('RGB'), dtype=np.float32)
    mask = np.array(Image.open(mask_path).convert('L'), dtype=np.float32)
    mask[mask == 255] = 1

    if self.transform is not None:
      augmentations = self.transform(image=image, mask = mask)

      image = augmentations['image']
      mask = augmentations['mask']

    return image, mask

def check_accuracy(loader, model, device='cuda'):
      num_correct = 0
      num_pixels = 0
      dice_score = 0

      model.eval()

      with torch.no_grad():
        for x, y in loader:
          x = x.to(device)
          y = y.to(device).unsqueeze(1)

          preds = torch.sigmoid(model(x))
          preds = (preds > 0.5).float()

          num_correct += (preds == y).sum()
          num_pixels += torch.numel(preds)
          dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)

      print(f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}")
      print(f"Dice score: {dice_score/len(loader)}")

      model.train()

def save_predictions_as_imgs(loader, model, folder="saved_images/", device="cuda"):
      model.eval()
      for idx, (x, y) in enumerate(loader):
          x = x.to(device=device)
          with torch.no_grad():
              preds = torch.sigmoid(model(x))
              preds = (preds > 0.5).float()
          torchvision.utils.save_image(
              preds, f"{folder}/pred_{idx}.png"
          )
          torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

      model.train()

# Utils

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader

def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
  train_ds = SegmentationDataset(
          image_dir=train_dir,
          mask_dir=train_maskdir,
          transform=train_transform,
      )

  train_loader = DataLoader(
          train_ds,
          batch_size=batch_size,
          num_workers=num_workers,
          pin_memory=pin_memory,
          shuffle=True,
      )

  val_ds = SegmentationDataset(
          image_dir=val_dir,
          mask_dir=val_maskdir,
          transform=val_transform,
      )

  val_loader = DataLoader(
          val_ds,
          batch_size=batch_size,
          num_workers=num_workers,
          pin_memory=pin_memory,
          shuffle=False,
      )
  return train_loader, val_loader

aasffsfaffffffffffffffffafjiuiui# Training

In [ ]:
import torch
import albumentations  as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

# Hyperparameters
LR = 1e-4
BATCH_SIZE = 32
EPOCHS = 50
WORKERS = 2
HEIGHT = 360
WIDTH = 240
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = ""
TRAIN_MASK_DIR = ""
TEST_IMG_DIR = ""
TEST_MASK_DIR = ""
DEVICE = 'cuda' if torch.cuda.is_available() else ' cpu'

In [ ]:
def train_fun(loader, model, optimizer, loss_fun, scaler):
  loop = tqdm(loader)

  for batch_idx, (data, targets) in enumerate(loop):
    data = data.to(device=DEVICE)
    targets = targets.float().unsqueeze(1).to(device=DEVICE)

    # forward
    with torch.cuda.amp.autocast():
      predictions = model(data)
      loss = loss_fun(predictions, targets)


    # backward
    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    # update tqdm loop
    loop.set_postfix(loss=loss.item)


train_transform = A.Compose(
      [
          A.Resize(height=HEIGHT, width=WIDTH),
          A.Rotate(limit=35, p=0.3),
          A.HorizontalFlip(p=0.2),
          A.VerticalFlip(p=0.1),
          A.Normalize(
              mean=[0.0, 0.0, 0.0],
              std=[1.0, 1.0, 1.0],
              max_pixel_value=255.0,
          ),
          ToTensorV2(),
      ],
  )

val_transforms = A.Compose(
      [
          A.Resize(height=HEIGHT, width=WIDTH),
          A.Normalize(
              mean=[0.0, 0.0, 0.0],
              std=[1.0, 1.0, 1.0],
              max_pixel_value=255.0,
          ),
          ToTensorV2(),
      ],
  )

model = UNet(in_channels=3, out_channels=1).to(DEVICE)
loss_fun = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

train_loader, val_loader = get_loaders(
      TRAIN_IMG_DIR,
      TRAIN_MASK_DIR,
      TEST_IMG_DIR,
      TEST_MASK_DIR,
      BATCH_SIZE,
      train_transform,
      val_transforms,
      WORKERS,
      PIN_MEMORY,
  )

if LOAD_MODEL:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)

scaler = torch.cuda.amp.GradScaler()

for epoch in range(EPOCHS):
  train_fun(train_loader, model, optimizer, loss_fun, scaler)

  # save model
  checkpoint = {
    "state_dict": model.state_dict(),
    "optimizer": optimizer.state_dict(),
  }

  save_checkpoint(checkpoint)

  # check accuracy
  check_accuracy(val_loader, model, device=DEVICE)

  # print some of predictions
  save_predictions_as_imgs(val_loader, model, folder="saved_images", device=DEVICE)

100%|██████████| 32/32 [00:47<00:00,  1.48s/it, loss=<built-in method item of Tensor object at 0x7aed955f1670>]


=> Saving checkpoint
Got 13185045/17280000 with acc 76.30
Dice score: 0.2066970318555832


100%|██████████| 32/32 [00:41<00:00,  1.30s/it, loss=<built-in method item of Tensor object at 0x7aed9673a930>]


=> Saving checkpoint
Got 14411395/17280000 with acc 83.40
Dice score: 0.6968648433685303


100%|██████████| 32/32 [00:42<00:00,  1.32s/it, loss=<built-in method item of Tensor object at 0x7aed95604fe0>]


...

=> Saving checkpoint
Got 15631906/17280000 with acc 90.46
Dice score: 0.8105065226554871


100%|██████████| 32/32 [00:41<00:00,  1.31s/it, loss=<built-in method item of Tensor object at 0x7aed955233d0>]


=> Saving checkpoint
Got 15552350/17280000 with acc 90.00
Dice score: 0.8187683820724487


100%|██████████| 32/32 [00:41<00:00,  1.31s/it, loss=<built-in method item of Tensor object at 0x7aed9556ba60>]


=> Saving checkpoint
Got 15506793/17280000 with acc 89.74
Dice score: 0.8148431181907654
